In [2]:
import numpy as np
import anndata as ad
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import anndata
import seaborn as sns
import scvelo as scv 
from scipy.stats import zscore
from dtne import *

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 数据准备
adata = l23_PC1_shifted
adata.X = np.nan_to_num(adata.X)

# 定义基因集
def get_common_genes(gene_set, slope_condition):
    return list(set(gene_set).intersection(
        V1_L23_PC1_df.query(slope_condition)['Gene']).intersection(adata.var_names))

genesets = {
    'RP_pos': get_common_genes(RP_gene_order_list, "NR_slope > 0"),
    'RP_neg': get_common_genes(RP_gene_order_list, "NR_slope < 0"),
    'RN_pos': get_common_genes(RN_gene_order_list, "NR_slope > 0"),
    'RN_neg': get_common_genes(RN_gene_order_list, "NR_slope < 0"),
    'NRP_pos': get_common_genes(NRP_gene_order_list, "NR_slope > 0"),
    'NRP_neg': get_common_genes(NRP_gene_order_list, "NR_slope < 0"),
    'NRN_pos': get_common_genes(NRN_gene_order_list, "NR_slope > 0"),
    'NRN_neg': get_common_genes(NRN_gene_order_list, "NR_slope < 0"),
}

slice_categories = ['NR1_2', 'NR2_2', 'DR1_2', 'DR2_2']
n_rows, n_cols = len(genesets), len(slice_categories)

# 创建绘图布局，关闭 constrained_layout 使用自定义调整
fig, axs = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))

# 用于存储颜色范围
global_min, global_max = float('inf'), float('-inf')

for row_idx, (geneset_name, geneset) in enumerate(genesets.items()):
    if not geneset:
        for ax in axs[row_idx, :]:
            ax.axis("off")
        continue

    slice_g_values = {}
    slice_gmin_gmax = {}
    vmax_vmin_diffs = {}

    # Step 1: 计算每个 slice 的 g 值并缩放
    for slice_category in slice_categories:
        slice_data = adata[adata.obs['slice'] == slice_category]
        if slice_data.n_obs > 0:
            g = slice_data[:, geneset].X.mean(axis=1).flatten()
            vmin, vmax = np.percentile(g, 5), np.percentile(g, 95)
            g_clipped = np.clip(g, vmin, vmax)
            g_min, g_max = np.min(g_clipped), np.max(g_clipped)
            g_norm = (g_clipped - g_min) / (g_max - g_min)
            slice_g_values[slice_category] = (g_norm, slice_data.obsm['stereo_rotated'][:, 0], slice_data.obsm['stereo_rotated'][:, 1])
            slice_gmin_gmax[slice_category] = (g_min, g_max)
            vmax_vmin_diffs[slice_category] = vmax - vmin
            global_min = min(global_min, g_clipped.min())
            global_max = max(global_max, g_clipped.max())

    # Step 2: 选择基准 slice（vmax - vmin 最大的 slice）
    base_slice = max(vmax_vmin_diffs, key=vmax_vmin_diffs.get)
    base_gmin, base_gmax = slice_gmin_gmax[base_slice]

    # Step 3: 调整其他 slice 的 g 值
    adjusted_g_values = {}
    for slice_category in slice_categories:
        g_norm, x, y = slice_g_values.get(slice_category, (None, None, None))
        g_min, g_max = slice_gmin_gmax.get(slice_category, (None, None))
        if g_norm is not None:
            # 使用基准 slice 的缩放公式调整 g 值
            g_adjusted = 0.5 + (g_norm - 0.5) * (g_max - g_min) / (base_gmax - base_gmin)
            adjusted_g_values[slice_category] = (g_adjusted, x, y)

    # Step 4: 绘制每个 slice 的图
    for col_idx, slice_category in enumerate(slice_categories):
        ax = axs[row_idx, col_idx]
        adjusted_g, x, y = adjusted_g_values.get(slice_category, (None, None, None))

        if adjusted_g is None:
            ax.axis("off")
            continue

        # 绘制散点图，使用基准 slice 的颜色范围 [0, 1]
        scatter = ax.scatter(
            x, y,
            c=adjusted_g, cmap='seismic', s=10, alpha=1.0, vmin=0, vmax=1
        )

        # 设置标题
#        ax.set_title(f"{slice_category} - {geneset_name}", fontsize=10)
        ax.axis("off")

# 添加全局颜色条
#cbar_ax = fig.add_axes([0.2, 0.05, 0.6, 0.02])  # [左, 下, 宽度, 高度]
#cbar = fig.colorbar(
#    plt.cm.ScalarMappable(norm=plt.Normalize(vmin=0, vmax=1), cmap='seismic'),
#    cax=cbar_ax,
#    orientation='horizontal'
#)
#cbar.set_label('Gene Expression (Adjusted)', fontsize=12)

# 添加总标题并调整位置
#plt.suptitle("Gene Expression with Slice-wise Clipping and Scaling (5%-95%)", fontsize=16, y=0.97)
plt.subplots_adjust(top=0.9, bottom=0.1)  # 调整布局，避免重叠
#plt.show()
output_path = "F:/spatial/V1_integration/NR_V/figures/V1_l23_adjusted_g.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()

print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/V1_integration/NR_V/figures/V1_l23_adjusted_g.pdf


In [37]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.nonparametric.smoothers_lowess import lowess
# 数据准备
adata = l23_PC1_shifted
adata.X = np.nan_to_num(adata.X)

# 定义基因集
def get_common_genes(gene_set, slope_condition):
    return list(set(gene_set).intersection(
        V1_L23_PC1_df.query(slope_condition)['Gene']).intersection(adata.var_names))

genesets = {
    'RP_pos': get_common_genes(RP_gene_order_list, "NR_slope > 0"),
    'RP_neg': get_common_genes(RP_gene_order_list, "NR_slope < 0"),
    'RN_pos': get_common_genes(RN_gene_order_list, "NR_slope > 0"),
    'RN_neg': get_common_genes(RN_gene_order_list, "NR_slope < 0"),
    'NRP_pos': get_common_genes(NRP_gene_order_list, "NR_slope > 0"),
    'NRP_neg': get_common_genes(NRP_gene_order_list, "NR_slope < 0"),
    'NRN_pos': get_common_genes(NRN_gene_order_list, "NR_slope > 0"),
    'NRN_neg': get_common_genes(NRN_gene_order_list, "NR_slope < 0"),
}

slice_categories = ['NR1_2', 'NR2_2', 'DR1_2', 'DR2_2']

# 创建 1列8行的图形布局
fig, axs = plt.subplots(8, 1, figsize=(4, 4 * len(genesets)), constrained_layout=True)
NR_color = '#E7241D'
DR_color = '#2372A9'
# 为每个 geneset 绘制一个折线图
for row_idx, (geneset_name, geneset) in enumerate(genesets.items()):
    ax = axs[row_idx]  # 获取子图

    if not geneset:
        ax.axis("off")
        continue

    slice_g_values = {}
    slice_gmin_gmax = {}
    slice_vmin_vmax = {}
    vmax_vmin_diffs = {}

    # Step 1: 计算每个 slice 的 g 值并缩放
    for slice_category in slice_categories:
        slice_data = adata[adata.obs['slice'] == slice_category]
        if slice_data.n_obs > 0:
            g = slice_data[:, geneset].X.mean(axis=1).flatten()
            vmin, vmax = np.percentile(g, 5), np.percentile(g, 95)
            g_clipped = np.clip(g, vmin, vmax)
            g_min, g_max = np.min(g_clipped), np.max(g_clipped)
            g_norm = (g_clipped - g_min) / (g_max - g_min)
            slice_g_values[slice_category] = (g_norm, slice_data.obs['digital_layer'])
            slice_gmin_gmax[slice_category] = (g_min, g_max)
            slice_vmin_vmax[slice_category] = (vmin, vmax)
            vmax_vmin_diffs[slice_category] = vmax - vmin

    # Step 2: 选择基准 slice（vmax - vmin 最大的 slice）
    base_slice = max(vmax_vmin_diffs, key=vmax_vmin_diffs.get)
    base_gmin, base_gmax = slice_gmin_gmax[base_slice]

    # Step 3: 调整其他 slice 的 g 值
    adjusted_g_values = {}
    for slice_category in slice_categories:
        g_norm, x = slice_g_values.get(slice_category, (None, None))
        g_min, g_max = slice_gmin_gmax.get(slice_category, (None, None))
        if g_norm is not None:
            # 使用基准 slice 的缩放公式调整 g 值
            g_adjusted = 0.5 + (g_norm - 0.5) * (g_max - g_min) / (base_gmax - base_gmin)
            adjusted_g_values[slice_category] = (g_adjusted, x)

    # Step 4: 合并 NR 和 DR 数据
    nr_g_values = []
    nr_x_values = []
    dr_g_values = []
    dr_x_values = []

    for slice_category in ['NR1_2', 'NR2_2']:
        if slice_category in adjusted_g_values:
            g, x = adjusted_g_values[slice_category]
            nr_g_values.append(g)
            nr_x_values.append(x)

    for slice_category in ['DR1_2', 'DR2_2']:
        if slice_category in adjusted_g_values:
            g, x = adjusted_g_values[slice_category]
            dr_g_values.append(g)
            dr_x_values.append(x)

    if nr_g_values:
        nr_g_values = np.concatenate(nr_g_values)
        nr_x_values = np.concatenate(nr_x_values)

    if dr_g_values:
        dr_g_values = np.concatenate(dr_g_values)
        dr_x_values = np.concatenate(dr_x_values)


#   smooth_frac = 0.66  # 越大曲线越平滑
    
    # Step 5: 绘制综合后的折线图
    for g_values, x_values, color, label in zip(
        [nr_g_values, dr_g_values],
        [nr_x_values, dr_x_values],
        [NR_color, DR_color],
        ['NR', 'DR']
    ):
        if g_values.size > 0:
            bins = np.arange(min(x_values), max(x_values), 0.25)
            bin_centers = 0.5 * (bins[1:] + bins[:-1])

            # 计算每个 bin 的均值和标准差
            binned_means = [np.mean(g_values[(x_values >= bins[i]) & (x_values < bins[i + 1])])
                            for i in range(len(bins) - 1)]
            binned_std = [np.std(g_values[(x_values >= bins[i]) & (x_values < bins[i + 1])])
                          for i in range(len(bins) - 1)]

            # LOWESS 会返回和 bin_centers 对应长度的平滑值
#            means_smooth = lowess(
#                       binned_means, 
#                       bin_centers, 
#                       frac=smooth_frac, 
#                       return_sorted=False
#             )
#            std_smooth = lowess(
#                       binned_std, 
#                       bin_centers, 
#                       frac=smooth_frac, 
#                       return_sorted=False
#             )

            # 绘制平滑后的曲线
            ax.plot(bin_centers, binned_means, color=color, label=label)

            # 绘制阴影：用平滑后的标准差 * 0.05
            ax.fill_between(
                 bin_centers,
                 np.array(binned_means)-np.array(binned_std) * 0.05,
                 np.array(binned_means)+np.array(binned_std) * 0.05,
                 color=color, alpha=0.1
             )
    # 设置图标题和标签
    ax.set_title(f"{geneset_name}", fontsize=14)
    ax.set_xlabel('Digital Layer', fontsize=12)
    ax.set_ylabel('Normalized Gene Expression', fontsize=12)

    ax.legend(title="Conditions", fontsize=10, loc="upper right")

# 添加总标题
plt.suptitle("Line Plots of Gene Expression across Conditions", fontsize=16, y=1.02)
#plt.show()
output_path = "F:/spatial/V1_integration/NR_V/figures/V1_l23_adjusted_lineplot.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/V1_integration/NR_V/figures/V1_l23_adjusted_lineplot.pdf


In [121]:
import scvelo as scv
import matplotlib.pyplot as plt
import numpy as np

# 设置数据
adata = l23_PC1_shifted
adata.X = np.nan_to_num(adata.X)# 替换为你自己的数据

# 目标基因列表
genes = ['Hadh', 'Hs3st1', 'Igfn1', 'Pcdh15', 'Sema3a', 'Egfem1', 'Fgf1', 
         'Igsf9b', 'Nell1', 'Cdh13', 'Epha6', 'Ntng1', 'Rorb', 'Trpc6']

# 设置 slice 分类
slice_categories = ['NR1_2', 'NR2_2', 'DR1_2', 'DR2_2']

# 创建子图布局，纵向显示不同gene，横向显示4个slice
fig, axes = plt.subplots(len(genes), len(slice_categories), figsize=(12, 2 * len(genes)))
# 为每个gene绘制子图
for i, gene in enumerate(genes):
    for j, slice_category in enumerate(slice_categories):
        ax = axes[i, j]  # 获取当前子图位置
        
        # 筛选当前 slice 的数据
        subset = adata[adata.obs['slice'] == slice_category, :]
        
        # 获取当前基因的表达数据
        gene_expression = np.asarray(subset[:, gene].X).flatten()

        
        # 绘制当前 gene 的散点图
        scv.pl.scatter(subset, basis='stereo_rotated', color=gene_expression, 
                       cmap='viridis', ax=ax, show=False, 
                       size=25, title=f'{slice_category} - {gene}',
                       colorbar=True)
        
        # 隐藏坐标轴标签，避免重复显示
        ax.set_xlabel('')
        ax.set_ylabel('')

# 设置统一的 colorbar
plt.subplots_adjust(hspace=0.4, wspace=0.5)
#plt.show()
output_path = "F:/spatial/V1_integration/NR_V/figures/V1_l23_TF_gene.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()

print(f"PDF saved as {output_path}")

Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.nonparametric.smoothers_lowess import lowess
adata = l23_PC1_shifted
# 目标基因
genes = ['Egfem1','Brinp3','Cdh13','Ntng1','Kirrel3','Trpm3','Nwd2','Cadps2']
# 设置条件（NR 和 DR）
conditions = ['NR', 'DR']
condition_colors = {
    'NR': '#E7241D',   
    'DR': '#2372A9'    
}
# 设置 bin 宽度
bin_width = 0.25

# 创建子图布局，纵向显示不同gene
fig, axes = plt.subplots(len(genes), 1, figsize=(3.2, 4 * len(genes)))

#smooth_frac = 0.66  # 越大曲线越平滑

# 为每个基因绘制折线图
for i, gene in enumerate(genes):
    ax = axes[i]  # 获取当前子图位置
    
    for condition in conditions:
        color = condition_colors[condition]
        # 获取当前条件的数据
        condition_data = l23_PC1_shifted[adata.obs['condition'] == condition]
        
        # 获取基因表达值和 digital_layer 数据
        gene_expression = condition_data[:, gene].X.toarray().flatten()
        digital_layer = condition_data.obs['digital_layer'].values
        
        # 创建 bins，范围从 min 到 max，步长为 bin_width
        bins = np.arange(min(digital_layer), max(digital_layer), bin_width)
        bin_centers = 0.5 * (bins[1:] + bins[:-1])
        
        # 计算每个 bin 的均值和标准差
        binned_means = [np.mean(gene_expression[(digital_layer >= bins[i]) & (digital_layer < bins[i + 1])])
                        for i in range(len(bins) - 1)]
        binned_std = [np.std(gene_expression[(digital_layer >= bins[i]) & (digital_layer < bins[i + 1])])
                      for i in range(len(bins) - 1)]

        # LOWESS 会返回和 bin_centers 对应长度的平滑值
#        means_smooth = lowess(
#                       binned_means, 
#                       bin_centers, 
#                       frac=smooth_frac, 
#                       return_sorted=False
#             )
#        std_smooth = lowess(
#                       binned_std, 
#                       bin_centers, 
#                       frac=smooth_frac, 
#                       return_sorted=False
#             )

        # 绘制折线图
        ax.plot(bin_centers, binned_means, color=color, label=condition)

        # 添加标准偏差阴影
        ax.fill_between(bin_centers,
                            np.array(binned_means) - np.array(binned_std) * 0.05,
                            np.array(binned_means) + np.array(binned_std) * 0.05,
                            color=color, alpha=0.1)  
    # 设置图标题和标签
    ax.set_title(f"Gene Expression for {gene}", fontsize=14)
    ax.set_xlabel('Digital Layer', fontsize=12)
    ax.set_ylabel('Gene Expression', fontsize=12)
    
    # 设置图例
    ax.legend(title="Conditions", fontsize=10, loc="upper right")

# 调整子图布局
plt.subplots_adjust(hspace=0.4)
#plt.show()
output_path = "F:/spatial/V1_integration/NR_V/figures/V1_l23_gene_lineplot.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/V1_integration/NR_V/figures/V1_l23_gene_lineplot.pdf
